# แบบฝึกหัดเขียนโปรแกรมครั้งที่ 2

## 3.เขียนโปรแกรมสำหรับเส้นโค้งการเรียนรู้เปรียบเทียบระหว่างแบบจำลองค่าคงที่ แบบจำลองเชิงเส้น แบบจำลองเชิงเส้นผ่านจุดกำเนิด และทดลองเพิ่มเติมด้วยการใส่สัญญาณรบกวน (Lecture หน้า 35-37)

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import random


In [11]:

def sin_pi(x):
    return np.sin(np.dot(np.pi, x))


In [12]:

def hypothesis(X,w):
    X_b = np.c_[np.ones((X.shape[0], 1)), X]
    return np.dot(X_b,w)


In [13]:

def hypothesis_no_origin(x, w):
    return w * x


In [14]:

def constant_model(x, c):
    return c * np.ones_like(x)


In [15]:

def normal_equation(X, y):
    return np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)


In [16]:

def generate_learning_curves(target_func, model_func, train_sizes, noise_level=0.0, n_iter=1000):
    val_x = np.linspace(-1, 1, 100)
    val_y = target_func(val_x) + np.random.normal(0, noise_level, val_x.shape)

    e_in = []
    e_out = []
    model_params = []

    for train_size in train_sizes:
        train_errors = []
        val_errors = []
        iteration_params = []

        for _ in range(n_iter):
            sam_x = np.array(random.choices(val_x, k=train_size))
            sam_y = target_func(sam_x) + np.random.normal(0, noise_level, sam_x.shape)

            if model_func == constant_model:
                model_param = np.mean(sam_y)
            else:
                X_b = np.c_[np.ones((sam_x.shape[0], 1)), sam_x] if model_func != hypothesis_no_origin else sam_x.reshape(-1, 1)
                model_param = normal_equation(X_b, sam_y)

            train_loss = np.mean((model_func(sam_x, model_param) - sam_y) ** 2)
            train_errors.append(train_loss)

            val_loss = np.mean((model_func(val_x, model_param) - val_y) ** 2)
            val_errors.append(val_loss)

            iteration_params.append(model_param)

        e_in.append(np.mean(train_errors))
        e_out.append(np.mean(val_errors))
        model_params.append(np.mean(iteration_params, axis=0))

    func_g = model_func(val_x, np.mean(model_params, axis=0))
    bias = np.mean((func_g - val_y)**2)

    return e_in, e_out, bias


In [17]:

def plot_learning_curves(train_sizes, e_in, e_out, bias, model_name):
    plt.figure(figsize=(12, 8))
    plt.plot(train_sizes, e_in, label=f'E_in (Cross-validation error)', color='b')
    plt.plot(train_sizes, e_out, label=f'E_out (Training error)', color='r')
    plt.axhline(y=bias, color='g', linestyle=':', label=f'Bias ({bias:.2f})')
    plt.xlabel("Training Set Size")
    plt.ylabel("Expected Error")
    plt.title(model_name)
    plt.legend()
    plt.grid(True)
    plt.show()

In [18]:

# Define training set sizes
train_sizes = range(2, 30, 2)

# Generate learning curves for different models with noise
noise_level = 0.3
e_in_const, e_out_const, bias_const = generate_learning_curves(sin_pi, constant_model, train_sizes, noise_level)
e_in_linear, e_out_linear, bias_linear = generate_learning_curves(sin_pi, hypothesis, train_sizes, noise_level)
e_in_no_origin, e_out_no_origin, bias_no_origin = generate_learning_curves(sin_pi, hypothesis_no_origin, train_sizes, noise_level)

# Plot learning curves
plt.figure(figsize=(12, 8))
plot_learning_curves(train_sizes, e_in_const, e_out_const, bias_const, 'Constant Model')
plot_learning_curves(train_sizes, e_in_linear, e_out_linear, bias_linear, 'Linear Model')
plot_learning_curves(train_sizes, e_in_no_origin, e_out_no_origin, bias_no_origin, 'Linear Model (No Origin)')



ValueError: shapes (2,2) and (1,) not aligned: 2 (dim 1) != 1 (dim 0)